In [ ]:
# section 1 load all the necessary modules and packages
import glob
import time
import netCDF4 as nc4
import numpy as np
import pandas as pd
import xarray as xr
# not neccessary for the function but for visualziation
import matplotlib.pyplot as plt
import warnings
import sys
import os
import platform
import shutil


In [ ]:
###########################
# replacing string function
###########################
def replace_string(file_in, file_out, replacement_dict):
    """
    Replace strings in a text file using a dictionary of replacements and save the modified content to a new file.

    Parameters:
        file_in (str): Path to the input text file.
        file_out (str): Path to the output text file.
        replacement_dict (dict): Dictionary of old strings as keys and their corresponding replacements as values.

    Returns:
        None
    """
    try:
        with open(file_in, "r+") as text_file:
            texts = text_file.read()
            for old_str, new_str in replacement_dict.items():
                texts = texts.replace(old_str, new_str)

        with open(file_out, "w") as text_file:
            text_file.write(texts)

        print("Strings replaced and saved successfully!")

    except Exception as e:
        print(f"An error occurred: {e}")
        
import xarray as xr

def replace_and_save_netcdf(input_file, output_file, variable_values):
    """
    Reads the input NetCDF file, replaces the specified variables with new values,
    and saves the modified data as a new NetCDF file.

    Parameters:
        input_file (str): Path to the input NetCDF file.
        output_file (str): Name of the output NetCDF file.
        variable_values (dict): Dictionary of variable names and their corresponding new values.

    Returns:
        None
    """
    try:
        # Read the input NetCDF file using xarray
        ds = xr.open_dataset(input_file)

        # Replace variables with new values
        for var_name, new_value in variable_values.items():
            if var_name in ds:
                ds[var_name].values[:] = new_value

        # Save the modified data to the output NetCDF file
        if os.path.isfile(output_file):
            os.remove(output_file)
        ds.to_netcdf(output_file)

        # Close the dataset
        ds.close()

        print(f"Data with modified variables saved to {output_file}")

    except Exception as e:
        print(f"Error: {e}")


def copy_folderA_to_folderB(path_org, path_target):
    """
    Copy the contents of folder A into folder B and remove any files or folders in folder B
    that are not present in folder A.

    Parameters:
        path_org (str): Path to the source folder A.
        path_target (str): Path to the target folder B.

    Returns:
        None
    """
    try:
        # Check if the target folder B exists; if not, copy the entire folder A to B
        if not os.path.isdir(path_target):
            shutil.copytree(path_org, path_target)
        else:
            # Remove any files or folders in folder B that are not present in folder A
            for root, dirs, files in os.walk(path_target):
                for name in files + dirs:
                    path = os.path.join(root, name)
                    if not os.path.exists(os.path.join(path_org, os.path.relpath(path, path_target))):
                        os.remove(path)

        print("Folder A contents copied to folder B with cleanup.")
    except Exception as e:
        print(f"An error occurred: {e}")
        

def gen_param(param_lim_low, param_lim_up, n):
    """
    Generate random parameter values within specified lower and upper limits.

    Parameters:
        param_lim_low (numpy.ndarray): 1D array of lower limits for each parameter.
        param_lim_up (numpy.ndarray): 1D array of upper limits for each parameter.
        n (int): Number of sets of random parameter values to generate.

    Returns:
        numpy.ndarray: A 2D array containing n sets of random parameter values, where each set is a row.

    Raises:
        ValueError: If param_lim_low and param_lim_up have different sizes or if any element in param_lim_low
                    is greater than the corresponding element in param_lim_up.
    """
    if len(param_lim_low) != len(param_lim_up):
        raise ValueError("param_lim_low and param_lim_up must have the same size.")

    if np.any(param_lim_low > param_lim_up):
        raise ValueError("Each element in param_lim_low must be less than or equal to the corresponding element in param_lim_up.")

    # Convert 1D arrays to 2D arrays of shape (n, num_parameters)
    param_lim_low_2D = np.atleast_2d(param_lim_low.flatten()).repeat(repeats=n, axis=0)
    param_lim_up_2D = np.atleast_2d(param_lim_up.flatten()).repeat(repeats=n, axis=0)

    # Generate random numbers between 0 and 1 for each parameter
    random = np.random.rand(n, len(param_lim_low.flatten()))

    # Scale and shift the random numbers to be within the specified parameter limits
    params = param_lim_low_2D + (param_lim_up_2D - param_lim_low_2D) * random

    return params


def slice_simulation(path, variables_to_keep, var_ID, target_IDs):
    
    for file_name in sorted(glob.glob(path)):

        # open netcdf file
        ds = xr.open_dataset(file_name)

        # Drop all variables except the ones to keep with dimension of seg, time, or seg and time
        variables_to_keep = ['IRFroutedRunoff', 'IRFvolume', 'time', 'reachID', 'time_bounds']
        ds = ds.drop_vars([var for var in ds.variables if var not in variables_to_keep])
        
        # get the location of lake victoria and its outflow and slice
        idx = np.where(np.isin(ds[var_ID].values, target_IDs))[0]
        ds = ds.isel(seg = idx)

        # save the file over the original file
        os.remove(file_name)
        ds.to_netcdf(file_name)
    
    



In [17]:
# calibration of the parameters

# generate the random parameters
names=['HYP_Erate_emr','HYP_Qrate_emr','scale_factor_Ep_temp','scale_factor_P_temp']
param_lim_low = np.array([1.00,  50.00, 0.60, 0.60])
param_lim_up  = np.array([3.00, 200.00, 1.40, 1.40])
param_num     = 10

params = gen_param(param_lim_low, param_lim_up, param_num)

# replace the first param with original parameter from agreed curve
params [0,0] = 2.01
params [0,1] = 66.3
params [0,2] = 1.000
params [0,3] = 1.000

# remove and create mizuRoute output folder
shutil.rmtree('../mizuRoute_output_all/', ignore_errors=True)
os.makedirs('../mizuRoute_output_all/')
np.savetxt("../mizuRoute_output_all/params.txt", params, fmt="%.5f")
param_df = pd.DataFrame(params, columns=names)
param_df.to_csv("../mizuRoute_output_all/params.csv")

# copy the set up to a new location
shutil.rmtree('../mizuRoute_sim/', ignore_errors=True)
copy_folderA_to_folderB('../mizuRoute/','../mizuRoute_sim/')
os.chdir('../mizuRoute_sim/')
os.system('chmod +x route_runoff.exe')

# create the restart file at the start of year 2000
# replace the values in the network topology for the HYPE formulation
variable_values = {'HYP_Erate_emr':2.01,'HYP_Qrate_emr':66}
replace_and_save_netcdf('./ancillary_data/Network_topology_lake_victoria.nc', './ancillary_data/Network_topology_lake_victoria.nc', variable_values)

# prepare the control files based on the replacemenet
replacement_dict = {'scale_factor_Ep_temp':str("{:.3f}".format(1)), 'scale_factor_P_temp':str("{:.3f}".format(1)), 'start_date_temp':'1979-01-01', 'end_date_temp':'1999-12-31'}
replace_string('./settings/lake_victoria_temp.control', './settings/lake_victoria.control', replacement_dict)

# execute the simulation
os.system('rm ./output/*.h.*.nc') # make sure output folder is empty, keep restart file with *.r.*nc
os.system('./route_runoff.exe ./settings/lake_victoria.control') # simulate mizuRoute

# transfer the historic benchamrk before 2000 to mizuRoute output
slice_simulation('./output/*.h.*.nc', ['IRFroutedRunoff', 'IRFvolume', 'time', 'reachID', 'time_bounds'],'reachID',[2062605, 7000016])
command = 'module load cdo; cdo mergetime ./output/*.h.*.nc ./output/sim_hist.nc'
os.system(command)
command = 'mv ./output/sim_hist.nc '+'../mizuRoute_output_all/'+'sim_hist.nc'
os.system(command)

#
m = 0
for row in params:
    
    #
    m = m + 1

    # replace the values in the network topology for the HYPE formulation
    variable_values = {'HYP_Erate_emr':row[0],'HYP_Qrate_emr':row[1]}
    replace_and_save_netcdf('./ancillary_data/Network_topology_lake_victoria.nc','./ancillary_data/Network_topology_lake_victoria.nc',variable_values)

    # prepare the control files based on the replacemenet
    replacement_dict = {'scale_factor_Ep_temp':str("{:.3f}".format(row[2])), 'scale_factor_P_temp':str("{:.3f}".format(row[3])),'start_date_temp':'2000-01-01', 'end_date_temp':'2020-12-31', '!<fname_state_in>':'<fname_state_in> '}
    replace_string('./settings/lake_victoria_temp.control','./settings/lake_victoria.control',replacement_dict)

    # execute the simulation
    os.system('rm ./output/*.h.*.nc') # make sure output folder is empty, keep restart file with *.r.*nc
    os.system('./route_runoff.exe ./settings/lake_victoria.control') # simulate mizuRoute

    # keep only two features from simulation, lake victoria and its outlet and save
    slice_simulation('./output/*.h.*.nc', ['IRFroutedRunoff', 'IRFvolume', 'time', 'reachID', 'time_bounds'],'reachID',[2062605, 7000016])
    command = 'module load cdo; cdo mergetime ./output/*.h.*.nc ./output/sim_'+str(m).zfill(5)+'.nc'
    os.system(command)
    command = 'mv ./output/sim_'+str(m).zfill(5)+'.nc '+'../mizuRoute_output_all/'+'sim_'+str(m).zfill(5)+'.nc'
    os.system(command)
    


[[  2.01        66.3          1.           1.        ]
 [  1.5787692   53.92912278   0.88704425   0.71209269]
 [  2.97917041 199.57686678   0.94395176   0.77507307]
 ...
 [  2.38248742  50.9746027    1.22292338   1.24973711]
 [  1.22439341 192.0164791    1.26579149   1.11612564]
 [  2.91982267 102.24993298   1.05579447   1.30582288]]


NameError: name 'll' is not defined

In [ ]:
# run the simulation with lake for the lake using Doll formulation
# prepare the control files based on the replacemenet
replacement_dict = {'scale_factor_Ep_temp':str("{:.3f}".format(1)),\
                    'scale_factor_P_temp':str("{:.3f}".format(1)),\
                    'start_date_temp':'1979-01-01',\
                    'end_date_temp':'2020-12-31',\
                    '<varname_lakeModelType>    lake_type':'<varname_lakeModelType>    lake'} # use the lake flag as param definding doll
replace_string('./settings/lake_victoria_temp.control','./settings/lake_victoria.control',replacement_dict)

# execute the simulation
os.system('rm ./output/*.h.*.nc') # make sure output folder is empty, keep restart file with *.r.*nc
os.system('./route_runoff.exe ./settings/lake_victoria.control') # simulate mizuRoute

# keep only two features from simulation, lake victoria and its outlet and save
slice_simulation('./output/*.h.*.nc', ['IRFroutedRunoff', 'IRFvolume', 'time', 'reachID', 'time_bounds'],'reachID',[2062605, 7000016])
command = 'module load cdo; cdo mergetime ./output/*.h.*.nc ./output/sim_Doll.nc'
os.system(command)
command = 'mv ./output/sim_Doll.nc'+'../mizuRoute_output_all/sim_Doll.nc'
os.system(command)


In [ ]:
# ###########################
# # get the evaporation for lake victoria 
# ###########################
# file_names = sorted(glob.glob('../mizuRoute/input/mizuRoute_mswep_v1_*_subset.nc'))

# datasets = [xr.open_dataset(file_name) for file_name in file_names]

# merged_dataset = xr.concat(datasets, dim='time')

# # number of years
# time_variable = merged_dataset['time']
# years = time_variable.dt.year
# years = np.array(years)
# n_years = np.unique(years)
# print(n_years)

# merged_dataset_slice = merged_dataset.sel(lat=-1.00, lon=33.00, method='nearest')

# print(merged_dataset_slice)
# merged_dataset_slice['evapw'].plot()

# average_evap = sum(np.array(merged_dataset_slice['evapw'][:]))/len(n_years)

# print(average_evap)

# # get the ratio for scale
# other_evap = np.array([average_evap, 1550, 1350, 1130, 1520, 1450, 1500, 1370, 1590, 1470, 1475, 1590, 1595, 1400])

# # creat the scale factors
# scales = other_evap/average_evap

# # the control to create the restart file
# m = 0
# job_string = ''
# for scale in scales:
    
#     case_name = str(m)
#     scale_value = str("{:.3f}".format(scale))
    
#     replace_string ('../mizuRoute/settings/lake_victoria_temp.control',\
#                     '../mizuRoute/settings/lake_victoria_'+case_name+'.control',\
#                     ['case_temp','scale_factor_temp'],\
#                     ['case_'+case_name+'_'+scale_value,scale_value])
    
#     replace_string ('../mizuRoute/settings/lake_victoria_'+case_name+'.control',\
#                     '../mizuRoute/settings/lake_victoria_'+case_name+'_restart.control',\
#                     ['!<fname_state_in>'],\
#                     ['<fname_state_in> '])
    
#     m +=1
    
#     # add to general string
#     job_string = job_string + '# case'+ case_name +'\n'
#     job_string = job_string + 'srun ./route_runoff.exe ./settings/lake_victoria_'+case_name+'.control \n' +\
#                               'mv ./output/case_'+case_name+'_'+scale_value+'.r.2014-01-01-00000.nc ./output/case_'+case_name+'_'+scale_value+'.r.1979-01-01-00000.nc \n'+\
#                               'srun ./route_runoff.exe ./settings/lake_victoria_'+case_name+'_restart.control \n'
#     job_string = job_string + '\n'
    
    

# shutil.copy('../mizuRoute/lake_victoria_temp.submit', '../mizuRoute/lake_victoria.submit')
# with open('../mizuRoute/lake_victoria.submit', "a") as file:
#     file.write(job_string)
    